##Auto-Correlation and SVM


In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import acf
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

In [ ]:
ticker_symbol = "^NSEI"
data = yf.download(ticker_symbol, start="2015-01-01", end="2023-07-05")
data.fillna(method='ffill', inplace=True)
data = data[['Close', 'Volume']]
train_data = data[data.index < '2023-01-01']
test_data = data[data.index >= '2023-01-01']



[*********************100%%**********************]  1 of 1 completed


In [ ]:
autocorr = acf(train_data['Close'], nlags=20)
print("Autocorrelation values:", autocorr)
def rolling_autocorr(series, window):
    return series.rolling(window).apply(lambda x: x.autocorr(), raw=False)
window_size = 20
train_data['Autocorr'] = rolling_autocorr(train_data['Close'], window=window_size)
test_data['Autocorr'] = rolling_autocorr(test_data['Close'], window=window_size)

train_data.dropna(inplace=True)
test_data.dropna(inplace=True)


Autocorrelation values: [1.         0.99799882 0.9959548  0.99389418 0.99184763 0.98982259
 0.987734   0.985653   0.98348687 0.98125462 0.97905277 0.9768531
 0.97468431 0.97239919 0.97017133 0.96800445 0.96581482 0.9636379
 0.96141274 0.95917654 0.95691178]


<ipython-input-7-7588d8e72b45>:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-7-7588d8e72b45>:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-7-7588d8e72b45>:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-7-7588d8e72b45>:10: SettingWithCopyWarning:


A value is trying to be 

In [ ]:
X_train = train_data[['Close', 'Volume', 'Autocorr']]
y_train = train_data['Close']
X_test = test_data[['Close', 'Volume', 'Autocorr']]
y_test = test_data['Close']


scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
svm_model = SVR(kernel='rbf')
svm_model.fit(X_train_scaled, y_train)
y_pred = svm_model.predict(X_test_scaled)

mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")
comparison_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred}, index=test_data.index)
print(comparison_df.head())


Mean Squared Error: 45936301.66577203
                  Actual     Predicted
Date                                  
2023-01-30  17648.949219  11172.692328
2023-01-31  17662.150391  11197.943374
2023-02-01  17616.300781  11204.574340
2023-02-02  17610.400391  11231.266905
2023-02-03  17854.050781  11235.703474


## Introducing BB, Moving average, EMA

In [ ]:
data['MA_200'] = data['Close'].rolling(window=200).mean()
data['MA_50'] = data['Close'].rolling(window=50).mean()
data['MA_20'] = data['Close'].rolling(window=20).mean()

# Calculate the Bollinger Bands (20-day MA, 2 standard deviations)
data['BB_upper'] = data['MA_20'] + (data['Close'].rolling(window=20).std() * 2)
data['BB_lower'] = data['MA_20'] - (data['Close'].rolling(window=20).std() * 2)

# Calculate the Exponential Moving Average (20-day EMA)
data['EMA_20'] = data['Close'].ewm(span=20, adjust=False).mean()


<ipython-input-10-efd7d06b77cf>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-10-efd7d06b77cf>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-10-efd7d06b77cf>:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
data.fillna(method='ffill', inplace=True)
data.dropna(inplace=True)
train_data = data[data.index < '2023-01-01']
test_data = data[data.index >= '2023-01-01']

def rolling_autocorr(series, window):
    return series.rolling(window).apply(lambda x: x.autocorr(), raw=False)
window_size = 20

train_data['Autocorr'] = rolling_autocorr(train_data['Close'], window=window_size)
test_data['Autocorr'] = rolling_autocorr(test_data['Close'], window=window_size)

train_data.dropna(inplace=True)
test_data.dropna(inplace=True)

<ipython-input-11-5d6238bf9b9a>:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-11-5d6238bf9b9a>:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-11-5d6238bf9b9a>:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-11-5d6238bf9b9a>:14: SettingWithCopyWarning:


A value is trying

In [ ]:
features = ['Close', 'Volume', 'Autocorr', 'MA_200', 'MA_50', 'MA_20', 'BB_upper', 'BB_lower', 'EMA_20']
X_train = train_data[features]
y_train = train_data['Close']
X_test = test_data[features]
y_test = test_data['Close']

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

svm_model = SVR(kernel='rbf')
svm_model.fit(X_train_scaled, y_train)

y_pred = svm_model.predict(X_test_scaled)

# Calculate the Mean Squared Error
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 41993485.79888248


## Lag feature and Techinal Indicator

In [ ]:
!pip install ta


  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29411 sha256=dc318c0d3f09958a5f8b9f6137f818176cf7fbca548231d6074ccd587586b8c3
  Stored in directory: /root/.cache/pip/wheels/5f/67/4f/8a9f252836e053e532c6587a3230bc72a4deb16b03a829610b
Successfully built ta


In [ ]:
import ta
data['RSI'] = ta.momentum.RSIIndicator(close=data['Close'], window=14).rsi()
macd = ta.trend.MACD(close=data['Close'])
data['MACD'] = macd.macd()
data['MACD_Signal'] = macd.macd_signal()
data['MACD_Diff'] = macd.macd_diff()
for lag in range(1, 6):
    data[f'Close_Lag_{lag}'] = data['Close'].shift(lag)
data.dropna(inplace=True)

In [ ]:
data.fillna(method='ffill', inplace=True)
data.dropna(inplace=True)
train_data = data[data.index < '2023-01-01']
test_data = data[data.index >= '2023-01-01']

def rolling_autocorr(series, window):
    return series.rolling(window).apply(lambda x: x.autocorr(), raw=False)

window_size = 20
train_data['Autocorr'] = rolling_autocorr(train_data['Close'], window=window_size)
test_data['Autocorr'] = rolling_autocorr(test_data['Close'], window=window_size)
train_data.dropna(inplace=True)
test_data.dropna(inplace=True)



<ipython-input-15-d16a31cf920c>:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-15-d16a31cf920c>:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-15-d16a31cf920c>:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-15-d16a31cf920c>:13: SettingWithCopyWarning:


A value is trying

In [ ]:
features = ['Close', 'Volume', 'Autocorr', 'MA_200', 'MA_50', 'MA_20', 'BB_upper', 'BB_lower', 'EMA_20',
            'RSI', 'MACD', 'MACD_Signal', 'MACD_Diff', 'Close_Lag_1', 'Close_Lag_2', 'Close_Lag_3', 'Close_Lag_4', 'Close_Lag_5']
X_train = train_data[features]
y_train = train_data['Close']
X_test = test_data[features]
y_test = test_data['Close']
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

svm_model = SVR(kernel='rbf')
svm_model.fit(X_train_scaled, y_train)
y_pred = svm_model.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 42498808.37773062


## Grid SearchCV and Gradient Booster

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
features = ['Close', 'Volume', 'MA_200', 'MA_50', 'MA_20', 'BB_upper', 'BB_lower', 'EMA_20',
            'RSI', 'MACD', 'MACD_Signal', 'MACD_Diff', 'Close_Lag_1', 'Close_Lag_2', 'Close_Lag_3', 'Close_Lag_4', 'Close_Lag_5']
X_train = train_data[features]
y_train = train_data['Close']
X_test = test_data[features]
y_test = test_data['Close']

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
gb_model = GradientBoostingRegressor()
param_grid = {
    'n_estimators': [100, 200],
    'learning_rate': [0.01, 0.1],
    'max_depth': [3, 5, 7],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 3]
}
grid_search = GridSearchCV(estimator=gb_model, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)
grid_search.fit(X_train_scaled, y_train)


best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

Fitting 5 folds for each of 48 candidates, totalling 240 fits
Mean Squared Error: 9405.327238594426


In [ ]:
data['Fibonacci_21'] = data['Close'].rolling(window=21).mean()
data['Fibonacci_34'] = data['Close'].rolling(window=34).mean()
data['Fibonacci_55'] = data['Close'].rolling(window=55).mean()


def hull_moving_average(data, window):
    half_window = int(window / 2)
    sqrt_window = int(np.sqrt(window))
    wma_half = 2 * data.rolling(window=half_window).mean()
    wma_full = data.rolling(window=window).mean()
    hma = (wma_half - wma_full).rolling(window=sqrt_window).mean()
    return hma

data['HMA_20'] = hull_moving_average(data['Close'], 20)

In [ ]:
for lag in range(1, 6):
    data[f'Close_Lag_{lag}'] = data['Close'].shift(lag)
data.dropna(inplace=True)
train_data = data[data.index < '2023-01-01']
test_data = data[data.index >= '2023-01-01']


In [ ]:
features = ['Close', 'Volume', 'MA_200', 'MA_50', 'MA_20', 'BB_upper', 'BB_lower', 'EMA_20',
            'RSI', 'MACD', 'MACD_Signal', 'MACD_Diff', 'Close_Lag_1', 'Close_Lag_2', 'Close_Lag_3',
            'Close_Lag_4', 'Close_Lag_5', 'Fibonacci_21', 'Fibonacci_34', 'Fibonacci_55', 'HMA_20']
X_train = train_data[features]
y_train = train_data['Close']
X_test = test_data[features]
y_test = test_data['Close']

# Standardize the feature
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Initialize Gradient Booster
gb_model = GradientBoostingRegressor()
param_grid = {
    'n_estimators': [100, 200],
    'learning_rate': [0.01, 0.1],
    'max_depth': [3, 5, 7],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 3]
}
grid_search = GridSearchCV(estimator=gb_model, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)
grid_search.fit(X_train_scaled, y_train)
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test_scaled)

# Calculate the Mean Squared Error
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

Fitting 5 folds for each of 48 candidates, totalling 240 fits
Mean Squared Error: 7892.258722360177
